# Your mission

# Import

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/MyDrive/Colab Notebooks/EI_ST4_G1/EI_TS_CS-20230526T084435Z-001/EI_TS_CS")

Mounted at /content/gdrive


In [ ]:
%run ./utils.ipynb

In [ ]:
FILE_PATH = "data/bilan.csv"
TARGET = "consommation_totale"
EXOGENEOUS= "Température normale lissée (°C)"

## Prepare the data

Define here the range of your train/test split

In [ ]:
def read_data(data_path : str = "data/bilan.csv") -> pd.DataFrame:
    df = pd.read_csv(data_path)
    df['horodate'] = pd.to_datetime(df['horodate']) # Convert 'horodate' column to datetime
    df = df.set_index('horodate') # Set 'horodate' as the index
    # Filter rows where the value of column "Mois" is 6
    #df = df[df['Mois'].isin([5,6])]
    hourly_avg = df.groupby('Mois')['consommation_totale'].mean().reset_index(name='consommation_totale')

    return df

df = read_data(FILE_PATH)

X_train = df[-10000:-100]
X_test = df[-100:]

In [ ]:
df

,Unnamed: 0,Mois,Injection RTE (W),Refoulement RTE (W),Pertes modélisées (W),consommation_totale,Consommation totale télérelevée (W),Consommation HTA télérelevée (W),Consommation totale profilée (W),Consommation HTA profilée (W),...,Production décentralisée profilée (W),Production photovoltaïque profilée (W),Production autre profilée (W),Température réalisée lissée (°C),Température normale lissée (°C),Production éolienne totale (W),Production photovoltaïque totale (W),Pseudo rayonnement,Consommation HTA totale (W),Soutirage net vers autres GRD (W)
horodate,,,,,,,,,,,,,,,,,,,,,
2019-07-03 22:00:00+02:00,0.0,7.0,3.383306e+10,2.725795e+09,2.286131e+09,3.475336e+10,1.174429e+10,1.168837e+10,2.300908e+10,1.551255e+09,...,2.154923e+07,1.414142e+07,1102801.0,22.9,20.6,5.076127e+09,1.471273e+07,67.0,1.323963e+10,241976206.0
2019-07-04 00:30:00+02:00,0.0,7.0,3.010544e+10,2.969232e+09,2.079282e+09,3.126325e+10,1.114404e+10,1.109063e+10,2.011922e+10,1.433882e+09,...,6.961098e+06,0.000000e+00,762042.0,22.2,20.3,5.328515e+09,4.912830e+05,73.0,1.252451e+10,223556213.0
2019-07-04 03:30:00+02:00,0.0,7.0,2.549278e+10,2.719259e+09,1.798193e+09,2.652442e+10,1.081601e+10,1.075762e+10,1.570841e+10,1.376614e+09,...,6.813437e+06,0.000000e+00,762042.0,21.9,20.0,4.603394e+09,4.877070e+05,83.0,1.213424e+10,164460823.0
2019-07-04 05:00:00+02:00,0.0,7.0,2.621578e+10,2.319118e+09,1.812301e+09,2.718336e+10,1.163813e+10,1.158143e+10,1.554523e+10,1.523139e+09,...,6.622984e+06,0.000000e+00,762042.0,21.7,19.8,4.169759e+09,4.905970e+05,89.0,1.310457e+10,180465603.0
2019-07-04 06:30:00+02:00,0.0,7.0,2.935876e+10,1.855103e+09,1.956309e+09,3.027574e+10,1.296523e+10,1.290623e+10,1.731052e+10,1.535346e+09,...,5.208770e+07,4.553791e+07,762042.0,21.5,19.7,3.727644e+09,9.889918e+07,92.0,1.444158e+10,215049347.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-19 10:30:00+01:00,0.0,3.0,4.738432e+10,4.935468e+09,4.457043e+09,5.241930e+10,1.926923e+10,1.592864e+10,3.315006e+10,4.064858e+08,...,1.018214e+09,1.010288e+09,2205471.0,6.7,8.7,8.539976e+09,3.006684e+09,29.0,1.633513e+10,375074101.0
2021-03-19 12:30:00+01:00,0.0,3.0,4.715819e+10,5.850612e+09,4.614302e+09,5.291407e+10,1.780583e+10,1.476675e+10,3.510824e+10,3.503636e+08,...,1.417184e+09,1.409243e+09,2205471.0,7.3,9.7,9.371482e+09,3.893055e+09,39.0,1.511712e+10,329267532.0
2021-03-19 13:00:00+01:00,0.0,3.0,4.595804e+10,6.111693e+09,4.507434e+09,5.174172e+10,1.772636e+10,1.472480e+10,3.401536e+10,3.530504e+08,...,1.444703e+09,1.436761e+09,2205471.0,7.4,10.0,9.426723e+09,4.015956e+09,42.0,1.507785e+10,317486066.0


# Modeling with ARIMAX


## Modeling
The following code allows ARIMAX modelling using the temperature as an exogeneous variable.

In [ ]:
X_test[TARGET]

horodate
2021-03-01 21:30:00+01:00    4.255785e+10
2021-03-01 22:00:00+01:00    4.250684e+10
2021-03-02 07:00:00+01:00    4.857131e+10
2021-03-02 10:00:00+01:00    4.947499e+10
2021-03-03 11:00:00+01:00    4.826795e+10
                                 ...     
2021-03-19 10:30:00+01:00    5.241930e+10
2021-03-19 12:30:00+01:00    5.291407e+10
2021-03-19 13:00:00+01:00    5.174172e+10
2021-03-19 13:30:00+01:00    5.054535e+10
2021-03-19 14:00:00+01:00    4.946106e+10
Name: consommation_totale, Length: 100, dtype: float64

In [ ]:
errors, predictions = evaluate_xgboost_model(
    np.array(X_train[TARGET]).reshape(-1, 1),
    np.array(X_test[TARGET]).reshape(-1,1),
    np.array(X_train[EXOGENEOUS]).reshape(-1,1),
    np.array(X_test[EXOGENEOUS]).reshape(-1,1),
    )


[09:09:42] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[09:09:43] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[09:09:43] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[09:09:43] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[09:09:43] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[09:09:43] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[09:09:43] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[09:09:43] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[09:09:43] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[09:09:43] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[09:09:43] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[09:09:43]

## Search for the best XGBOOST



In [ ]:
params_grid = {
    'max_depth': [1,2,3,4,5,6,7,8,9,10],
    'learning_rate':[0.01, 1, 0.01],
    'n_estimators': [k for k in range (100)]
}

best_cfg, best_score = xgboost_grid_search(    np.array(X_train[TARGET]).reshape(-1, 1),
    np.array(X_test[TARGET]).reshape(-1,1),
    np.array(X_train[EXOGENEOUS]).reshape(-1,1),
    np.array(X_test[EXOGENEOUS]).reshape(-1,1),
                                            params_grid)

Streaming output truncated to the last 5000 lines.
Parameters: { "n_estimators" } are not used.

XGBoost dict_keys(['max_depth', 'learning_rate', 'n_estimators'])=(10, 0.88, 50) RMSE=5303803758.586393
[09:51:09] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

XGBoost dict_keys(['max_depth', 'learning_rate', 'n_estimators'])=(10, 0.88, 51) RMSE=5303803758.586393
[09:51:09] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

XGBoost dict_keys(['max_depth', 'learning_rate', 'n_estimators'])=(10, 0.88, 52) RMSE=5303803758.586393
[09:51:09] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

XGBoost dict_keys(['max_depth', 'learning_rate', 'n_estimators'])=(10, 0.88, 53) RMSE=5303803758.586393
[09:51:09] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

XGBoost dict_keys(['max_depth', 'learning_rate', 'n_estimators'])=(10, 0.88, 54) RMSE=5303803758.586393
[09:51:09] WARNING: ../

In [ ]:
print(best_cfg, best_score)

{'max_depth': 1, 'learning_rate': 0.98, 'n_estimators': 0} 4757002406.216887


## Visualization
To have a better view on the difference between true and predict values, we visualize them by plotting both the signals.

In [ ]:
# prepare the dataset for plotting
df_reset = df.reset_index()
predict_date = df_reset["horodate"]
df_predict = pd.DataFrame(zip(predict_date[-100:],
                              predictions, X_test[TARGET].values),
                          columns=["date", "predict", "true"])

In [ ]:
df_predict

,date,predict,true
0,2021-03-01 21:30:00+01:00,4.408238e+10,4.255785e+10
1,2021-03-01 22:00:00+01:00,4.554487e+10,4.250684e+10
2,2021-03-02 07:00:00+01:00,4.488915e+10,4.857131e+10
3,2021-03-02 10:00:00+01:00,4.306426e+10,4.947499e+10
4,2021-03-03 11:00:00+01:00,4.244330e+10,4.826795e+10
...,...,...,...
94,2021-03-18 20:30:00+01:00,3.941800e+10,4.974261e+10
95,2021-03-19 10:30:00+01:00,3.849543e+10,5.241930e+10
96,2021-03-19 12:30:00+01:00,3.869817e+10,5.291407e+10
97,2021-03-19 13:00:00+01:00,3.879138e+10,5.174172e+10


In [ ]:
A=df_predict['predict']
B=df_predict['true']

In [ ]:
A

0     4.408238e+10
1     4.554487e+10
2     4.488915e+10
3     4.306426e+10
4     4.244330e+10
          ...     
94    3.941800e+10
95    3.849543e+10
96    3.869817e+10
97    3.879138e+10
98    3.783073e+10
Name: predict, Length: 99, dtype: float32

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score
import statsmodels.api as sm
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBRegressor


model = XGBRegressor(max_depth=1,learning_rate=1,n_estimators=0 )
model_fit=model.fit(A,B)

predictions = model_fit.predict(B)
mae = mean_absolute_error(A,B)
rmse = mean_squared_error(A,B, squared=False)
r2 = r2_score(A,B)
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

AttributeError: ignored

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_predict["date"], y=df_predict["predict"], name="predict"))
fig.add_trace(go.Scatter(x=df_predict["date"], y=df_predict["true"], name="true"))

fig.update_layout(title="Predictions vs true values")

# Modeling with other models

Try other other models : random forest, xgboost ...